In [1]:
from src.train.ptuning_train import train
import src.benchmarks as benchmarks
import src.prompt.mbpp as mbpp
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import src.config as config
import torch

Skipping import of cpp extensions due to incompatible torch version 2.9.0+cu128 for torchao version 0.14.0         Please see GitHub issue #2919 for more info


In [2]:
train()

13:42:28 | INFO | Load Base Model: Qwen/Qwen3-0.6B
13:42:30 | INFO | Setup P-Tuning...
trainable params: 20,480 || all params: 596,070,400 || trainable%: 0.0034
13:42:30 | INFO | Tokenizing...
13:42:35 | INFO | Start Training...


Step,Training Loss
10,19.180900
20,12.645400
30,9.876600
40,6.807400
50,6.186200
60,5.759800
70,5.279900
80,5.003700
90,5.176300
100,4.963400


13:45:42 | INFO | Saved to ./models/qwen3-0.6b-ptuning


(PeftModelForCausalLM(
   (base_model): Qwen3ForCausalLM(
     (model): Qwen3Model(
       (embed_tokens): Embedding(151936, 1024)
       (layers): ModuleList(
         (0-27): 28 x Qwen3DecoderLayer(
           (self_attn): Qwen3Attention(
             (q_proj): Linear4bit(in_features=1024, out_features=2048, bias=False)
             (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
             (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
             (o_proj): Linear4bit(in_features=2048, out_features=1024, bias=False)
             (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
             (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
           )
           (mlp): Qwen3MLP(
             (gate_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
             (up_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
             (down_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
             (act_fn

In [2]:
print("⏳ Загружаем базу...")
base_model = AutoModelForCausalLM.from_pretrained(
    config.MODEL_PATH,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_PATH, trust_remote_code=True)

print("🔗 Подключаем P-Tuning адаптер...")
# Путь должен совпадать с тем, куда сохранил train_ptuning.py
ptuning_path = config.PTUNING_MODEL_PATH
model = PeftModel.from_pretrained(base_model, ptuning_path)

print("✅ Готово к тестам!")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


⏳ Загружаем базу...
🔗 Подключаем P-Tuning адаптер...
✅ Готово к тестам!


In [3]:
prompt = (
    "<|im_start|>user\n"
    "Write a python function to reverse a string.<|im_end|>\n"
    "<|im_start|>assistant\n"
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024, # Даем место на подумать

        # === ЛЕКАРСТВО ОТ ЗАЦИКЛИВАНИЯ ===
        do_sample=True,        # Включаем сэмплирование (помогает выйти из цикла)
        temperature=0.1,       # Но держим очень низким, чтобы код был точным
        top_p=0.9,             # Немного свободы
        repetition_penalty=1.5, # <--- ГЛАВНОЕ: Штраф 20% за повторы
        # ================================

        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

print("-" * 30)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("-" * 30)

/opt/conda/lib/python3.11/site-packages/peft/peft_model.py:2141: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


------------------------------
user
Write a python function to reverse a string.
assistant
<think>

</think>


```python
def reversed_string(s):
    return s[::-1]
```
<tool_call>

Okay, let's see what the user is asking for here.

The task requires me to write an efficient Python program that can take any input and returns its reversal as output in one line of code.
I need to make sure I understand all parts correctly before proceeding with my response.)

Let’s break down your request:

- The first part: "write a python function" - this means we are supposed to create such-a-function (function) which takes no arguments but performs some operation on it or gives back something else?

Wait, maybe there was more context? Let me check again...

Ah yes! So according to their instructions:
"A.py file should contain exactly 2 lines."

So perhaps they want us to implement two functions?
But wait — how could you do that if each assistant has only three words per turn?"

Nope... Maybe just focu

In [4]:
import torch
from tqdm.auto import tqdm
import src.prompt.mbpp as mbpp
import src.test_execute as test_execute
import src.metrics as metrics_utils
import re

def extract_code_clean(text):
    pattern = r"```(?:python)?\n(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match: return match.group(1).strip()
    if "def " in text: return text
    return ""

def calculate_entropy_transformers_single(scores, sequences, input_len, batch_idx):
    """
    Считает энтропию для ОДНОГО элемента из батча.
    scores: кортеж тензоров (Steps, Batch, Vocab)
    """
    generated_ids = sequences[batch_idx, input_len:]
    log_probs_sum = 0.0
    count = 0

    for i, step_logits in enumerate(scores):
        if i >= len(generated_ids): break

        # step_logits имеет форму [Batch, Vocab]
        # Берем логиты нашего конкретного сэмпла
        sample_logits = step_logits[batch_idx]

        step_logprobs = torch.nn.functional.log_softmax(sample_logits, dim=-1)
        chosen_token_id = generated_ids[i]
        token_logprob = step_logprobs[chosen_token_id].item()

        log_probs_sum += token_logprob
        count += 1

    if count == 0: return 0.0
    return - (log_probs_sum / count)

def run_ptuning_mbpp(dataset_split, model, tokenizer, batch_size=8, limit=None):
    print(f"🚀 Запуск P-Tuning бенчмарка (Batched Transformers)...")

    # === ВАЖНО: ДЛЯ ГЕНЕРАЦИИ ПАДДИНГ СЛЕВА ===
    tokenizer.padding_side = "left"
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    # ==========================================

    metrics_all = []
    outputs_text = []

    dataset = dataset_split
    if limit:
        dataset = dataset.select(range(limit))

    # Цикл по батчам
    for i in tqdm(range(0, len(dataset), batch_size), desc="Benchmarking"):
        # 1. Формируем батч текстов
        batch_slice = dataset.select(range(i, min(i + batch_size, len(dataset))))
        prompts_text = [
            mbpp.build_prompt(ex, tokenizer, train=False)["text"]
            for ex in batch_slice
        ]

        # 2. Токенизация батча (с паддингом)
        inputs = tokenizer(
            prompts_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048
        ).to("cuda")

        input_len = inputs.input_ids.shape[1]

        # 3. ГЕНЕРАЦИЯ (Сразу для всей пачки)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=True,        # Включаем сэмплирование (помогает выйти из цикла)
                temperature=0.1,       # Но держим очень низким, чтобы код был точным
                top_p=0.9,             # Немного свободы
                repetition_penalty=1.5, # <--- ГЛАВНОЕ: Штраф 20% за повторы
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                return_dict_in_generate=True,
                output_scores=True
            )

        # 4. Обработка результатов (в цикле, но это быстро)
        batch_sequences = outputs.sequences
        batch_scores = outputs.scores

        for j, seq in enumerate(batch_sequences):
            # Декодинг
            full_text = tokenizer.decode(seq, skip_special_tokens=True)
            raw_answer = full_text.split("assistant")[-1].strip()

            # Чистка кода
            code_to_test = extract_code_clean(raw_answer)
            outputs_text.append(code_to_test)

            # Тесты
            example = batch_slice[j]
            tests = example["test_list"]
            passed, total = test_execute.run_mbpp_tests_for_sample(code_to_test, tests)

            # Энтропия
            entropy = calculate_entropy_transformers_single(batch_scores, batch_sequences, input_len, j)

            met = {
                "pass@1": 1 if passed == total else 0,
                "%passed": passed / total if total > 0 else 0,
                "entropy": entropy
            }
            metrics_all.append(met)

    return metrics_all, outputs_text

In [5]:
# Загружаем датасет (validation или test)
# Используем твой же mbpp.py
mbpp_data = mbpp.get_dataset()["test"] # или "test"

# Запускаем (ограничим 20 примерами для быстрой проверки, убери limit=20 для полного прогона)
metrics_result, codes = run_ptuning_mbpp(mbpp_data, model, tokenizer, batch_size=4, limit=None)

# Агрегируем результаты твоей функцией
final_stats = metrics_utils.aggregate_metrics(metrics_result)

print("\n" + "="*30)
print(f"📊 ИТОГОВЫЕ МЕТРИКИ P-TUNING:")
print(f"Pass@1:    {final_stats['mean_pass@1']:.2%}")
print(f"Avg Passed:{final_stats['mean_%_passed']:.2%}")
print(f"Samples:   {final_stats['total_samples']}")
print("="*30)

🚀 Запуск P-Tuning бенчмарка (Batched Transformers)...


Benchmarking:   0%|          | 0/65 [00:00<?, ?it/s]

KeyboardInterrupt: 